In [1]:
import pycaret
from pycaret.regression import *
from pycaret.classification import *
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
import scipy.stats as stats
from tableone import TableOne

In [2]:
pre = pd.read_csv(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\FDATA_Pre_2B_GAUS_NORM_RESAMPLE000_5Sig_2023-12-17-19-39-33_radiomics_features_ALL.csv", index_col=0) 
pre.set_index('pt_no', inplace=True)
current_pre = pre.columns.tolist()
new_pre = {col: 'pre_' + col for col in current_pre}
pre = pre.rename(columns=new_pre)
pre

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,pre_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,pre_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,pre_wavelet-LLL_glszm_ZoneEntropy,pre_wavelet-LLL_glszm_ZonePercentage,pre_wavelet-LLL_glszm_ZoneVariance,pre_wavelet-LLL_ngtdm_Busyness,pre_wavelet-LLL_ngtdm_Coarseness,pre_wavelet-LLL_ngtdm_Complexity,pre_wavelet-LLL_ngtdm_Contrast,pre_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,1.484027,33602.207506,-3.203427e-16,0.028649,16.896669,1.650207,1.435364,0.050513,1.451836,0.848727,...,5.151790,0.020648,2.521641,0.000432,2.845642e+07,9.015553,0.006481,0.170626,0.000521,0.029926
17815426,2.498474,17859.937894,3.275463e-01,0.084472,16.839751,2.816649,2.373878,0.139768,2.454317,0.963370,...,2.745954,0.090834,3.517816,0.006705,1.881134e+05,19.008323,0.002101,5.205460,0.019972,0.033623
20109646,1.867500,160782.348255,-3.203427e-16,0.115694,4.985230,1.992746,1.675709,0.221762,1.805688,0.762951,...,3.295786,0.033080,4.901244,0.000994,3.646460e+07,28.580566,0.001058,0.616141,0.005491,0.017261
22320836,1.952697,118591.566401,6.798011e-02,0.065774,26.780779,2.320580,1.877198,0.061034,1.901065,0.842189,...,5.047231,0.026280,4.297276,0.000924,3.283239e+07,15.015660,0.002133,0.337529,0.000162,0.028521
32221501,2.204831,30703.734710,4.964888e-01,0.076908,17.502032,2.391268,2.084067,0.121619,2.146633,0.854982,...,8.966514,0.014361,3.076103,0.002722,1.249836e+06,29.483228,0.001257,3.622199,0.011019,0.020562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,2.694805,13784.733946,8.584567e-02,0.071341,24.932035,3.403854,2.593937,0.073887,2.611643,1.458692,...,1.496849,0.196455,3.511085,0.009300,1.844272e+05,3.602884,0.016283,0.432528,0.000814,0.082850
56124435,2.403171,81778.369047,1.490141e-01,0.073894,33.752847,2.604287,2.318845,0.067155,2.343169,1.033921,...,4.021003,0.058208,4.292493,0.003033,3.886449e+06,16.606470,0.001619,1.329660,0.001298,0.041413
56131318,1.982925,44892.814442,2.641421e-01,0.050950,20.583522,2.235937,1.905351,0.079641,1.944605,0.941089,...,6.721493,0.040658,4.416044,0.004559,2.243085e+06,8.564359,0.002479,1.251746,0.001425,0.063484


In [3]:
delta = pd.read_csv(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\FDATA_Pre_2B_GAUS_NORM_RESAMPLE000_5Sig_2023-12-17-19-39-33_radiomics_features_ALL.csv", index_col=0) 
current_delta = delta.columns.tolist()
new_delta = {col: 'delta_' + col for col in current_delta}
delta = delta.rename(columns=new_delta)
delta

,delta_exponential_firstorder_90Percentile,delta_exponential_firstorder_Energy,delta_exponential_firstorder_Entropy,delta_exponential_firstorder_InterquartileRange,delta_exponential_firstorder_Kurtosis,delta_exponential_firstorder_Maximum,delta_exponential_firstorder_Mean,delta_exponential_firstorder_MeanAbsoluteDeviation,delta_exponential_firstorder_Median,delta_exponential_firstorder_Minimum,...,delta_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength
exponential_firstorder_10Percentile,,,,,,,,,,,,,,,,,,,,,
1.404239,1.484027,33602.207506,-3.203427e-16,0.028649,16.896669,1.650207,1.435364,0.050513,1.451836,0.848727,...,5.151790,0.020648,2.521641,0.000432,2.845642e+07,9.015553,0.006481,0.170626,0.000521,0.029926
2.229674,2.498474,17859.937894,3.275463e-01,0.084472,16.839751,2.816649,2.373878,0.139768,2.454317,0.963370,...,2.745954,0.090834,3.517816,0.006705,1.881134e+05,19.008323,0.002101,5.205460,0.019972,0.033623
0.944226,1.867500,160782.348255,-3.203427e-16,0.115694,4.985230,1.992746,1.675709,0.221762,1.805688,0.762951,...,3.295786,0.033080,4.901244,0.000994,3.646460e+07,28.580566,0.001058,0.616141,0.005491,0.017261
1.794773,1.952697,118591.566401,6.798011e-02,0.065774,26.780779,2.320580,1.877198,0.061034,1.901065,0.842189,...,5.047231,0.026280,4.297276,0.000924,3.283239e+07,15.015660,0.002133,0.337529,0.000162,0.028521
1.982924,2.204831,30703.734710,4.964888e-01,0.076908,17.502032,2.391268,2.084067,0.121619,2.146633,0.854982,...,8.966514,0.014361,3.076103,0.002722,1.249836e+06,29.483228,0.001257,3.622199,0.011019,0.020562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2.492946,2.694805,13784.733946,8.584567e-02,0.071341,24.932035,3.403854,2.593937,0.073887,2.611643,1.458692,...,1.496849,0.196455,3.511085,0.009300,1.844272e+05,3.602884,0.016283,0.432528,0.000814,0.082850
2.223602,2.403171,81778.369047,1.490141e-01,0.073894,33.752847,2.604287,2.318845,0.067155,2.343169,1.033921,...,4.021003,0.058208,4.292493,0.003033,3.886449e+06,16.606470,0.001619,1.329660,0.001298,0.041413
1.835408,1.982925,44892.814442,2.641421e-01,0.050950,20.583522,2.235937,1.905351,0.079641,1.944605,0.941089,...,6.721493,0.040658,4.416044,0.004559,2.243085e+06,8.564359,0.002479,1.251746,0.001425,0.063484


In [4]:
ypCR_data = pd.read_excel(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\clinical_immunologic.xlsx", index_col=0) 
ypCR_data.set_index('pt_no', inplace=True)
ypCR_data = ypCR_data[ypCR_data['Pathol_available'] == 1]
ypCR_data

,name,Ineligible,Ineligible_2019,Pathol_available,Hospital,note,base.fu,date.dx,dob,age,...,note.1,other.malig,pre_PDL1_CPS,pre_PDL1_IC,pre_CD3_num_Pos_mm,pre_CD8_num_Pos_mm,post_PDL1_CPS,post_PDL1_IC,post_CD3_num_Pos_mm,post_CD8_num_Pos_mm
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,정종일,0,0,1.0,1,NaN,2016-11-17,2016-10-24,1946-03-05,69.731507,...,NaN,NaN,5.0,5.0,366.0532,51.9522,10.0,10.0,3171.7243,890.4297
17815426,김진영,1,0,1.0,1,NaN,2021-07-30,2021-07-08,1955-01-04,65.660274,...,NaN,NaN,50.0,40.0,637.6734,374.4912,80.0,80.0,1532.5386,1298.4060
20109646,서대현,0,0,1.0,1,NaN,2018-07-09,2018-05-18,1951-06-20,66.134247,...,NaN,NaN,0.5,0.5,60.6479,39.7833,0.0,0.0,100.5308,43.6339
22320836,최흥우,0,0,1.0,1,NaN,2016-06-07,2016-05-18,1947-07-11,67.961644,...,"diaphragmatic hernia reduction (2020/3/17, 201...",NaN,10.0,10.0,608.9312,140.8317,10.0,10.0,1176.5166,1516.6479
32221501,박남훈,0,0,1.0,1,NaN,2010-10-11,2010-09-17,1939-10-16,70.013699,...,NaN,NaN,1.0,1.0,504.0535,64.6015,10.0,10.0,1245.4099,1155.8683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,최정옥,1,0,1.0,1,Previous malignancy: Breast ca (하지만 1995년이라 ok),2021-10-26,2021-09-29,1961-11-27,59.093151,...,NaN,NaN,10.0,5.0,1152.7963,471.6934,100.0,70.0,1477.2669,868.1072
56124435,김정자,1,0,1.0,1,NaN,2021-12-03,2021-11-09,1963-01-13,58.082192,...,NaN,NaN,0.0,0.0,145.0536,31.3657,10.0,10.0,479.3238,389.0902
56131318,홍성배,1,0,1.0,1,NaN,2021-11-23,2021-11-10,1951-05-20,69.542466,...,NaN,NaN,10.0,10.0,5283.4120,5773.6300,50.0,50.0,4583.2514,4448.2354


In [5]:
data = pd.concat([pre, delta], axis=1)
data

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,delta_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength
1.308546e+07,1.484027,33602.207506,-3.203427e-16,0.028649,16.896669,1.650207,1.435364,0.050513,1.451836,0.848727,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.781543e+07,2.498474,17859.937894,3.275463e-01,0.084472,16.839751,2.816649,2.373878,0.139768,2.454317,0.963370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.010965e+07,1.867500,160782.348255,-3.203427e-16,0.115694,4.985230,1.992746,1.675709,0.221762,1.805688,0.762951,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.232084e+07,1.952697,118591.566401,6.798011e-02,0.065774,26.780779,2.320580,1.877198,0.061034,1.901065,0.842189,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.222150e+07,2.204831,30703.734710,4.964888e-01,0.076908,17.502032,2.391268,2.084067,0.121619,2.146633,0.854982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2.492946e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.496849,0.196455,3.511085,0.009300,1.844272e+05,3.602884,0.016283,0.432528,0.000814,0.082850
2.223602e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.021003,0.058208,4.292493,0.003033,3.886449e+06,16.606470,0.001619,1.329660,0.001298,0.041413
1.835408e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.721493,0.040658,4.416044,0.004559,2.243085e+06,8.564359,0.002479,1.251746,0.001425,0.063484
2.269771e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.053635,0.081890,4.411769,0.004695,1.617504e+06,7.366517,0.005814,0.562301,0.000261,0.044813


In [6]:
data['post_PDL1_CPS'] = ypCR_data['post_PDL1_CPS']
data

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength,post_PDL1_CPS
1.308546e+07,1.484027,33602.207506,-3.203427e-16,0.028649,16.896669,1.650207,1.435364,0.050513,1.451836,0.848727,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
1.781543e+07,2.498474,17859.937894,3.275463e-01,0.084472,16.839751,2.816649,2.373878,0.139768,2.454317,0.963370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0
2.010965e+07,1.867500,160782.348255,-3.203427e-16,0.115694,4.985230,1.992746,1.675709,0.221762,1.805688,0.762951,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2.232084e+07,1.952697,118591.566401,6.798011e-02,0.065774,26.780779,2.320580,1.877198,0.061034,1.901065,0.842189,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
3.222150e+07,2.204831,30703.734710,4.964888e-01,0.076908,17.502032,2.391268,2.084067,0.121619,2.146633,0.854982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2.492946e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.196455,3.511085,0.009300,1.844272e+05,3.602884,0.016283,0.432528,0.000814,0.082850,NaN
2.223602e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.058208,4.292493,0.003033,3.886449e+06,16.606470,0.001619,1.329660,0.001298,0.041413,NaN
1.835408e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.040658,4.416044,0.004559,2.243085e+06,8.564359,0.002479,1.251746,0.001425,0.063484,NaN
2.269771e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.081890,4.411769,0.004695,1.617504e+06,7.366517,0.005814,0.562301,0.000261,0.044813,NaN


In [7]:
# bin_labels = [0, 1, 2]
# bins = [-1, 1, 10, float('inf')]
# post['encoded_PDL1'] = pd.cut(post['post_PDL1_CPS'], bins=bins, labels=bin_labels)
# post
conditions = [
    (data['post_PDL1_CPS'] >= 0) & (data['post_PDL1_CPS'] < 1),
    (data['post_PDL1_CPS'] >= 1) & (data['post_PDL1_CPS'] <= 10),
    (data['post_PDL1_CPS'] > 10)
]
choices = [0, 1, 2]
data['encoded_PDL1'] = np.select(conditions, choices, default=np.nan)
data

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength,post_PDL1_CPS,encoded_PDL1
1.308546e+07,1.484027,33602.207506,-3.203427e-16,0.028649,16.896669,1.650207,1.435364,0.050513,1.451836,0.848727,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,1.0
1.781543e+07,2.498474,17859.937894,3.275463e-01,0.084472,16.839751,2.816649,2.373878,0.139768,2.454317,0.963370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,2.0
2.010965e+07,1.867500,160782.348255,-3.203427e-16,0.115694,4.985230,1.992746,1.675709,0.221762,1.805688,0.762951,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2.232084e+07,1.952697,118591.566401,6.798011e-02,0.065774,26.780779,2.320580,1.877198,0.061034,1.901065,0.842189,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,1.0
3.222150e+07,2.204831,30703.734710,4.964888e-01,0.076908,17.502032,2.391268,2.084067,0.121619,2.146633,0.854982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2.492946e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.511085,0.009300,1.844272e+05,3.602884,0.016283,0.432528,0.000814,0.082850,NaN,NaN
2.223602e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.292493,0.003033,3.886449e+06,16.606470,0.001619,1.329660,0.001298,0.041413,NaN,NaN
1.835408e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.416044,0.004559,2.243085e+06,8.564359,0.002479,1.251746,0.001425,0.063484,NaN,NaN
2.269771e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.411769,0.004695,1.617504e+06,7.366517,0.005814,0.562301,0.000261,0.044813,NaN,NaN


In [8]:
data.drop(columns=['post_PDL1_CPS'], inplace=True)
data

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength,encoded_PDL1
1.308546e+07,1.484027,33602.207506,-3.203427e-16,0.028649,16.896669,1.650207,1.435364,0.050513,1.451836,0.848727,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1.781543e+07,2.498474,17859.937894,3.275463e-01,0.084472,16.839751,2.816649,2.373878,0.139768,2.454317,0.963370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2.010965e+07,1.867500,160782.348255,-3.203427e-16,0.115694,4.985230,1.992746,1.675709,0.221762,1.805688,0.762951,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2.232084e+07,1.952697,118591.566401,6.798011e-02,0.065774,26.780779,2.320580,1.877198,0.061034,1.901065,0.842189,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3.222150e+07,2.204831,30703.734710,4.964888e-01,0.076908,17.502032,2.391268,2.084067,0.121619,2.146633,0.854982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2.492946e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.196455,3.511085,0.009300,1.844272e+05,3.602884,0.016283,0.432528,0.000814,0.082850,NaN
2.223602e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.058208,4.292493,0.003033,3.886449e+06,16.606470,0.001619,1.329660,0.001298,0.041413,NaN
1.835408e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.040658,4.416044,0.004559,2.243085e+06,8.564359,0.002479,1.251746,0.001425,0.063484,NaN
2.269771e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.081890,4.411769,0.004695,1.617504e+06,7.366517,0.005814,0.562301,0.000261,0.044813,NaN


In [9]:
feature = data.drop(columns='encoded_PDL1')
target = data['encoded_PDL1']
list_numerical = feature.columns.to_list()
list_numerical

['pre_exponential_firstorder_90Percentile',
 'pre_exponential_firstorder_Energy',
 'pre_exponential_firstorder_Entropy',
 'pre_exponential_firstorder_InterquartileRange',
 'pre_exponential_firstorder_Kurtosis',
 'pre_exponential_firstorder_Maximum',
 'pre_exponential_firstorder_Mean',
 'pre_exponential_firstorder_MeanAbsoluteDeviation',
 'pre_exponential_firstorder_Median',
 'pre_exponential_firstorder_Minimum',
 'pre_exponential_firstorder_Range',
 'pre_exponential_firstorder_RobustMeanAbsoluteDeviation',
 'pre_exponential_firstorder_RootMeanSquared',
 'pre_exponential_firstorder_Skewness',
 'pre_exponential_firstorder_TotalEnergy',
 'pre_exponential_firstorder_Uniformity',
 'pre_exponential_firstorder_Variance',
 'pre_exponential_glcm_Autocorrelation',
 'pre_exponential_glcm_ClusterProminence',
 'pre_exponential_glcm_ClusterShade',
 'pre_exponential_glcm_ClusterTendency',
 'pre_exponential_glcm_Contrast',
 'pre_exponential_glcm_Correlation',
 'pre_exponential_glcm_DifferenceAverage',

In [10]:
feature

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,delta_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength
1.308546e+07,1.484027,33602.207506,-3.203427e-16,0.028649,16.896669,1.650207,1.435364,0.050513,1.451836,0.848727,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.781543e+07,2.498474,17859.937894,3.275463e-01,0.084472,16.839751,2.816649,2.373878,0.139768,2.454317,0.963370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.010965e+07,1.867500,160782.348255,-3.203427e-16,0.115694,4.985230,1.992746,1.675709,0.221762,1.805688,0.762951,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.232084e+07,1.952697,118591.566401,6.798011e-02,0.065774,26.780779,2.320580,1.877198,0.061034,1.901065,0.842189,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.222150e+07,2.204831,30703.734710,4.964888e-01,0.076908,17.502032,2.391268,2.084067,0.121619,2.146633,0.854982,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2.492946e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.496849,0.196455,3.511085,0.009300,1.844272e+05,3.602884,0.016283,0.432528,0.000814,0.082850
2.223602e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.021003,0.058208,4.292493,0.003033,3.886449e+06,16.606470,0.001619,1.329660,0.001298,0.041413
1.835408e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.721493,0.040658,4.416044,0.004559,2.243085e+06,8.564359,0.002479,1.251746,0.001425,0.063484
2.269771e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.053635,0.081890,4.411769,0.004695,1.617504e+06,7.366517,0.005814,0.562301,0.000261,0.044813


In [11]:
# def custom_zscore(column):
#     if column.std() == 0:
#         return (column - column.mean())  # For columns with zero variance, mean-centered values
#     else:
#         return (column - column.mean()) / column.std()

# # Apply custom_zscore to each column
# feature = feature.apply(custom_zscore)
# feature

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.linear_model import Lasso, LassoCV
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [13]:
# lasso 를 위한 데이터 나누기 
X_train, X_test, y_train, y_test = train_test_split(feature, target, shuffle=True, test_size = 0.2, stratify=target, random_state=2023)

ValueError: Input y contains NaN.

In [ ]:
X_train

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,delta_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
41375060,1.913437,16892.002232,1.343667e-02,0.056566,22.919338,2.044471,1.838553,0.067367,1.869661,0.897939,...,3.712660,0.045608,1.525231,0.008755,-3.789789e+06,44.931031,-0.007340,0.149927,0.000608,-0.043247
37567637,2.086047,9293.345336,8.503110e-01,0.070566,20.438827,2.149139,1.999038,0.078855,2.036572,1.000295,...,-2.767768,0.218069,-0.639354,-0.008054,1.952179e+07,-7.709710,0.020640,-1.845684,-0.011488,0.005915
49087653,1.806664,23083.640462,-3.203427e-16,0.068359,18.502705,1.917878,1.719973,0.070291,1.748438,0.936480,...,-2.105801,-0.015490,-0.851733,-0.002741,3.583023e+07,-10.389029,0.004871,-2.040553,-0.006333,0.051593
46695156,1.733236,7984.640391,1.925557e-01,0.045929,266.121365,11.138778,1.728045,0.095784,1.694074,1.048333,...,-13.651342,-0.014117,-1.188595,-0.010801,2.913572e+06,19.467073,-0.007118,-6.084096,0.011340,-1.422193
54502130,2.461475,22585.726458,1.653282e-01,0.052003,40.364190,2.730718,2.390674,0.060950,2.413734,1.017602,...,4.448422,-0.009817,-0.764656,-0.002872,5.336440e+06,36.094545,-0.013138,2.579153,0.002983,-0.164958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47047248,3.179595,42314.189622,3.927374e-01,0.654389,5.508685,5.148650,2.502102,0.398477,2.383120,1.818160,...,-3.931039,-0.116855,-0.175324,-0.002942,8.552021e+05,62.221669,-0.000776,-1.385447,0.009070,-0.024865
37509338,2.263698,4558.466574,8.426243e-02,0.086331,3.544149,2.475461,2.189351,0.055713,2.201546,1.975546,...,0.639735,0.010700,3.000000,0.000345,3.086606e+06,6.305412,-999999.990676,0.292578,0.000762,0.048651
50740947,1.944689,55447.935894,3.301658e-02,0.031311,106.157388,2.114963,1.911616,0.023902,1.917436,1.013016,...,-2.627177,-0.051192,1.712897,-0.000592,1.976091e+07,7.873596,-0.012838,0.428536,0.000517,-0.141541


In [ ]:
X_test

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,delta_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
42748515,2.201648,39069.292027,2.315166e-01,0.069860,43.469720,2.427748,2.125740,0.054280,2.137784,0.941167,...,5.961157,0.018577,1.017141,0.003584,-2.221883e+06,-51.965991,0.000283,-0.162587,-0.005935,0.017544
22320836,1.952697,118591.566401,6.798011e-02,0.065774,26.780779,2.320580,1.877198,0.061034,1.901065,0.842189,...,1.833348,-0.012926,-0.787715,0.000246,-1.443771e+07,55.996462,-0.001601,1.278782,0.008829,-0.021723
56131813,2.398622,46511.517902,1.060820e-01,0.057122,44.788503,3.312703,2.334439,0.051460,2.349742,1.118705,...,-0.082819,0.011403,-0.241786,-0.003475,1.278301e+07,229.369008,-0.005528,1.112013,0.008574,-0.042068
37244770,1.901291,83628.209721,5.243291e-03,0.045579,4.274601,2.023777,1.859029,0.029094,1.864095,1.559196,...,0.922495,-0.518547,1.870035,0.000564,-4.186273e+07,0.184928,-0.013964,0.359789,0.000599,0.021699
49344356,1.685784,46081.954347,2.363807e-03,0.042663,11.579530,2.241984,1.604154,0.091589,1.653138,0.857264,...,4.413640,-0.007191,-0.055995,0.000013,5.950454e+05,9.670855,-0.000673,0.152857,0.001436,-0.009085
42120720,2.354662,15279.297108,4.372304e-01,0.102524,12.463237,2.819949,2.212760,0.161193,2.307719,0.959301,...,-5.945015,-0.025132,0.873087,-0.007040,7.828450e+06,13.235655,-0.001086,-5.639784,-0.011643,-0.070389
51524302,2.448859,60249.579189,3.580777e-02,0.068568,29.175909,3.479093,2.382891,0.047403,2.393265,1.387185,...,0.659891,-0.047359,0.520184,0.000124,3.477882e+06,11.190002,-0.013543,0.144951,0.000093,-0.062825
47451087,1.746566,40093.208534,-3.203427e-16,0.054608,19.606255,1.842178,1.683773,0.054437,1.707863,1.028719,...,-2.242555,0.214013,0.691563,0.000913,6.537215e+05,15.180199,0.000160,-0.369953,-0.001383,-0.016435
53004114,2.314032,16411.127041,1.428297e-01,0.081631,17.944753,2.575424,2.225085,0.059852,2.232936,1.355104,...,3.255458,-0.092059,1.468045,-0.000285,1.216261e+06,29.426239,-0.026447,3.760034,0.002288,-0.298484


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train[list_numerical]) 

X_train[list_numerical] = scaler.transform(X_train[list_numerical])

X_test[list_numerical] = scaler.transform(X_test[list_numerical])

In [ ]:
X_train

,pre_exponential_firstorder_90Percentile,pre_exponential_firstorder_Energy,pre_exponential_firstorder_Entropy,pre_exponential_firstorder_InterquartileRange,pre_exponential_firstorder_Kurtosis,pre_exponential_firstorder_Maximum,pre_exponential_firstorder_Mean,pre_exponential_firstorder_MeanAbsoluteDeviation,pre_exponential_firstorder_Median,pre_exponential_firstorder_Minimum,...,delta_wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,delta_wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,delta_wavelet-LLL_glszm_ZoneEntropy,delta_wavelet-LLL_glszm_ZonePercentage,delta_wavelet-LLL_glszm_ZoneVariance,delta_wavelet-LLL_ngtdm_Busyness,delta_wavelet-LLL_ngtdm_Coarseness,delta_wavelet-LLL_ngtdm_Complexity,delta_wavelet-LLL_ngtdm_Contrast,delta_wavelet-LLL_ngtdm_Strength
pt_no,,,,,,,,,,,,,,,,,,,,,
41375060,-1.056040,-0.546057,-0.935698,-0.354476,-0.161911,-0.537157,-1.067749,-0.347070,-1.058970,-0.758498,...,1.041185,0.430590,1.417986,2.970664,-0.773458,0.121528,0.112509,0.284331,0.144845,0.095695
37567637,-0.527841,-0.790511,1.808028,-0.255055,-0.185794,-0.470398,-0.505671,-0.212172,-0.479077,-0.438568,...,-0.729295,1.800501,-0.849912,-1.609575,1.157863,0.044854,0.112509,-0.674535,-1.007998,0.341880
49087653,-1.382771,-0.346868,-0.979751,-0.270728,-0.204436,-0.617901,-1.483059,-0.312736,-1.480131,-0.638032,...,-0.548443,-0.054723,-1.072427,-0.161731,2.508987,0.040952,0.112509,-0.768167,-0.516646,0.570617
46695156,-1.607464,-0.832613,-0.348449,-0.430017,2.179736,5.263398,-1.454787,-0.013381,-1.669006,-0.288418,...,-3.702733,-0.043819,-1.425367,-2.357935,-0.218097,0.084438,0.112509,-2.711037,1.167611,-6.809536
54502130,0.620992,-0.362886,-0.437716,-0.386881,0.006055,-0.099454,0.865980,-0.422430,0.831282,-0.384472,...,1.242198,-0.009661,-0.981194,-0.197446,-0.017367,0.108657,0.112509,1.451543,0.371115,-0.513785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47047248,2.818484,0.271791,0.307854,3.891071,-0.329547,1.442759,1.256242,3.541132,0.724921,2.117791,...,-1.047106,-0.859898,-0.363735,-0.216729,-0.388629,0.146713,0.112509,-0.453397,0.951292,0.187747
37509338,0.015781,-0.942835,-0.703493,-0.143093,-0.348463,-0.262262,0.160874,-0.483925,0.094085,2.609726,...,0.201649,0.153311,2.963144,0.679053,-0.203761,0.065268,-8.888194,0.352873,0.159518,0.555886
50740947,-0.960405,0.694312,-0.871505,-0.533831,0.639539,-0.492196,-0.811854,-0.857484,-0.892987,-0.398807,...,-0.690885,-0.338318,1.614610,0.423637,1.177674,0.067552,0.112509,0.418199,0.136157,-0.396522


In [ ]:
n_alphas = 100
alphas = np.logspace(-4, -1.2, n_alphas)
print(alphas)
lasso_cv = LassoCV(alphas = alphas, random_state=2023, max_iter=1000000, cv=KFold(n_splits=5), verbose=True, n_jobs=-1).fit(X_train, y_train)

[1.00000000e-04 1.07726669e-04 1.16050353e-04 1.25017180e-04
 1.34676844e-04 1.45082878e-04 1.56292952e-04 1.68369191e-04
 1.81378522e-04 1.95393040e-04 2.10490414e-04 2.26754313e-04
 2.44274868e-04 2.63149179e-04 2.83481846e-04 3.05385551e-04
 3.28981682e-04 3.54401009e-04 3.81784403e-04 4.11283621e-04
 4.43062146e-04 4.77296092e-04 5.14175183e-04 5.53903799e-04
 5.96702113e-04 6.42807312e-04 6.92474907e-04 7.45980152e-04
 8.03619571e-04 8.65712598e-04 9.32603347e-04 1.00466252e-03
 1.08228947e-03 1.16591440e-03 1.25600075e-03 1.35304777e-03
 1.45759330e-03 1.57021671e-03 1.69154217e-03 1.82224203e-03
 1.96304065e-03 2.11471831e-03 2.27811560e-03 2.45413806e-03
 2.64376119e-03 2.84803587e-03 3.06809418e-03 3.30515567e-03
 3.56053412e-03 3.83564481e-03 4.13201240e-03 4.45127933e-03
 4.79521496e-03 5.16572536e-03 5.56486388e-03 5.99484250e-03
 6.45804416e-03 6.95703587e-03 7.49458302e-03 8.07366466e-03
 8.69749003e-03 9.36951631e-03 1.00934679e-02 1.08733567e-02
 1.17135050e-02 1.261856

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
...................................................................................................................................................................................................................................................................................................................................................[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    1.6s remaining:    2.4s
.................................................................................................................................................................[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.5s finished


In [ ]:
print('R squared training set', round(lasso_cv.score(X_train, y_train)*100, 2))
print('R squared test set', round(lasso_cv.score(X_test, y_test)*100, 2))

R squared training set 60.74
R squared test set 7.94


In [ ]:
best_alpha = lasso_cv.alpha_
print("Best alpha parameter:", best_alpha)

Best alpha parameter: 0.10140528875638684


In [ ]:
feature_list = lasso_cv.feature_names_in_[lasso_cv.coef_ != 0]
print(len(feature_list), feature_list)

30 ['pre_log-sigma-1-5-mm-3D_glszm_GrayLevelNonUniformityNormalized'
 'pre_log-sigma-1-5-mm-3D_glszm_SizeZoneNonUniformityNormalized'
 'pre_log-sigma-2-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis'
 'pre_original_shape_Maximum2DDiameterColumn'
 'pre_squareroot_firstorder_Mean'
 'pre_squareroot_glrlm_ShortRunLowGrayLevelEmphasis'
 'pre_squareroot_glszm_LargeAreaHighGrayLevelEmphasis'
 'pre_wavelet-HHH_firstorder_RobustMeanAbsoluteDeviation'
 'pre_wavelet-HLH_glszm_GrayLevelNonUniformityNormalized'
 'pre_wavelet-LHH_firstorder_InterquartileRange'
 'pre_wavelet-LHH_glcm_Autocorrelation'
 'delta_log-sigma-1-0-mm-3D_glszm_LargeAreaLowGrayLevelEmphasis'
 'delta_log-sigma-1-5-mm-3D_firstorder_Energy'
 'delta_log-sigma-1-5-mm-3D_firstorder_TotalEnergy'
 'delta_log-sigma-1-5-mm-3D_gldm_DependenceVariance'
 'delta_log-sigma-2-0-mm-3D_ngtdm_Busyness'
 'delta_log-sigma-2-5-mm-3D_glszm_ZoneEntropy'
 'delta_original_shape_LeastAxisLength' 'delta_square_glcm_Correlation'
 'delta_square_glrlm_LongRunH

In [ ]:
lasso_data = feature[feature_list]
lasso_data

,pre_log-sigma-1-5-mm-3D_glszm_GrayLevelNonUniformityNormalized,pre_log-sigma-1-5-mm-3D_glszm_SizeZoneNonUniformityNormalized,pre_log-sigma-2-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis,pre_original_shape_Maximum2DDiameterColumn,pre_squareroot_firstorder_Mean,pre_squareroot_glrlm_ShortRunLowGrayLevelEmphasis,pre_squareroot_glszm_LargeAreaHighGrayLevelEmphasis,pre_wavelet-HHH_firstorder_RobustMeanAbsoluteDeviation,pre_wavelet-HLH_glszm_GrayLevelNonUniformityNormalized,pre_wavelet-LHH_firstorder_InterquartileRange,...,delta_square_glszm_LowGrayLevelZoneEmphasis,delta_wavelet-HHH_glszm_ZoneEntropy,delta_wavelet-HHL_firstorder_Median,delta_wavelet-HHL_firstorder_Uniformity,delta_wavelet-HLH_glcm_ClusterShade,delta_wavelet-HLH_glcm_JointEnergy,delta_wavelet-HLL_firstorder_Energy,delta_wavelet-LLH_glszm_GrayLevelNonUniformityNormalized,delta_wavelet-LLH_glszm_SmallAreaEmphasis,delta_wavelet-LLL_glszm_SizeZoneNonUniformityNormalized
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,0.537190,0.123967,1.600125,43.657760,1.403847,0.087218,4.942979e+08,0.005046,0.520000,0.014027,...,0.000,-0.584963,-0.000650,8.139985e-05,0.005214,0.001723,-197.647973,0.007812,0.217912,-0.063673
17815426,0.580000,0.220000,0.604641,33.241540,3.644572,0.040037,1.200941e+07,0.014072,0.555556,0.041206,...,0.250,2.340159,0.000748,-1.701829e-04,0.004970,-0.000932,3219.113800,-0.002484,-0.252679,-0.124241
20109646,0.549861,0.182825,0.658829,73.061618,2.105729,0.108852,4.450030e+08,0.014033,0.663265,0.047805,...,0.000,0.000000,0.000028,3.085807e-05,0.001019,-0.000249,-2598.354714,-0.004853,-0.059095,0.044766
22320836,0.615912,0.171468,1.062641,48.010416,2.966077,0.031767,5.526197e+08,0.012025,0.625000,0.031887,...,0.000,0.000000,-0.000426,5.733739e-05,0.003937,-0.004728,-1988.693499,0.019497,-0.040512,0.038058
32221501,0.680000,0.245000,0.167926,44.598206,2.415741,0.062051,9.938098e+07,0.008163,0.625000,0.020100,...,0.000,-0.084963,0.000362,-4.333477e-05,-0.003499,0.005952,-695.601258,0.007982,-0.178836,0.057469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,0.555556,0.166667,0.005520,22.022716,4.487890,0.092725,2.310989e+06,0.015137,0.555556,0.043639,...,0.600,0.584963,0.001787,-5.198404e-04,-0.013734,-0.000008,-169.814090,0.018409,0.092998,0.065063
56124435,0.916824,0.232514,0.735408,48.764741,4.478155,0.035925,7.400588e+07,0.012610,0.520000,0.037374,...,0.000,0.584963,0.000059,-4.211582e-07,-0.001348,-0.001687,-433.785447,0.072068,0.087433,0.005844
56131318,0.571938,0.160523,0.717403,39.446166,4.330703,0.044694,6.091609e+07,0.015411,0.555556,0.044196,...,0.000,1.222192,0.000279,-5.044634e-05,0.001844,0.001527,3111.624309,-0.000907,0.144587,0.003868


In [ ]:
final_data = pd.concat([lasso_data, target], axis=1)
final_data

,pre_log-sigma-1-5-mm-3D_glszm_GrayLevelNonUniformityNormalized,pre_log-sigma-1-5-mm-3D_glszm_SizeZoneNonUniformityNormalized,pre_log-sigma-2-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis,pre_original_shape_Maximum2DDiameterColumn,pre_squareroot_firstorder_Mean,pre_squareroot_glrlm_ShortRunLowGrayLevelEmphasis,pre_squareroot_glszm_LargeAreaHighGrayLevelEmphasis,pre_wavelet-HHH_firstorder_RobustMeanAbsoluteDeviation,pre_wavelet-HLH_glszm_GrayLevelNonUniformityNormalized,pre_wavelet-LHH_firstorder_InterquartileRange,...,delta_wavelet-HHH_glszm_ZoneEntropy,delta_wavelet-HHL_firstorder_Median,delta_wavelet-HHL_firstorder_Uniformity,delta_wavelet-HLH_glcm_ClusterShade,delta_wavelet-HLH_glcm_JointEnergy,delta_wavelet-HLL_firstorder_Energy,delta_wavelet-LLH_glszm_GrayLevelNonUniformityNormalized,delta_wavelet-LLH_glszm_SmallAreaEmphasis,delta_wavelet-LLL_glszm_SizeZoneNonUniformityNormalized,encoded_PDL1
pt_no,,,,,,,,,,,,,,,,,,,,,
13085456,0.537190,0.123967,1.600125,43.657760,1.403847,0.087218,4.942979e+08,0.005046,0.520000,0.014027,...,-0.584963,-0.000650,8.139985e-05,0.005214,0.001723,-197.647973,0.007812,0.217912,-0.063673,1.0
17815426,0.580000,0.220000,0.604641,33.241540,3.644572,0.040037,1.200941e+07,0.014072,0.555556,0.041206,...,2.340159,0.000748,-1.701829e-04,0.004970,-0.000932,3219.113800,-0.002484,-0.252679,-0.124241,2.0
20109646,0.549861,0.182825,0.658829,73.061618,2.105729,0.108852,4.450030e+08,0.014033,0.663265,0.047805,...,0.000000,0.000028,3.085807e-05,0.001019,-0.000249,-2598.354714,-0.004853,-0.059095,0.044766,0.0
22320836,0.615912,0.171468,1.062641,48.010416,2.966077,0.031767,5.526197e+08,0.012025,0.625000,0.031887,...,0.000000,-0.000426,5.733739e-05,0.003937,-0.004728,-1988.693499,0.019497,-0.040512,0.038058,1.0
32221501,0.680000,0.245000,0.167926,44.598206,2.415741,0.062051,9.938098e+07,0.008163,0.625000,0.020100,...,-0.084963,0.000362,-4.333477e-05,-0.003499,0.005952,-695.601258,0.007982,-0.178836,0.057469,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55941790,0.555556,0.166667,0.005520,22.022716,4.487890,0.092725,2.310989e+06,0.015137,0.555556,0.043639,...,0.584963,0.001787,-5.198404e-04,-0.013734,-0.000008,-169.814090,0.018409,0.092998,0.065063,2.0
56124435,0.916824,0.232514,0.735408,48.764741,4.478155,0.035925,7.400588e+07,0.012610,0.520000,0.037374,...,0.584963,0.000059,-4.211582e-07,-0.001348,-0.001687,-433.785447,0.072068,0.087433,0.005844,1.0
56131318,0.571938,0.160523,0.717403,39.446166,4.330703,0.044694,6.091609e+07,0.015411,0.555556,0.044196,...,1.222192,0.000279,-5.044634e-05,0.001844,0.001527,3111.624309,-0.000907,0.144587,0.003868,2.0


In [ ]:
clinical_train = pd.read_csv(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\2FDATA_clinical_train.csv", index_col=0) 
clinical_test = pd.read_csv(r"C:\Users\user\Desktop\vsc\Severance_Hospital\12.19_정현도_alpha_feature\pre\2FDATA_clinical_test.csv", index_col=0) 
clinical_train.drop(columns=['TD', 'RT.tech_IMRT', 'location'], inplace=True)
clinical_test.drop(columns=['TD', 'RT.tech_IMRT', 'location'], inplace=True)
clinical_train

,encoded_PDL1
pt_no,
51127213,1.0
38924846,0.0
56241796,1.0
40815882,1.0
33908849,0.0
...,...
47181472,2.0
49087653,1.0
49588132,0.0


In [ ]:
rad_features_list = final_data.columns.to_list()
rad_features_list.remove('encoded_PDL1')
rad_features_list

['pre_log-sigma-1-5-mm-3D_glszm_GrayLevelNonUniformityNormalized',
 'pre_log-sigma-1-5-mm-3D_glszm_SizeZoneNonUniformityNormalized',
 'pre_log-sigma-2-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis',
 'pre_original_shape_Maximum2DDiameterColumn',
 'pre_squareroot_firstorder_Mean',
 'pre_squareroot_glrlm_ShortRunLowGrayLevelEmphasis',
 'pre_squareroot_glszm_LargeAreaHighGrayLevelEmphasis',
 'pre_wavelet-HHH_firstorder_RobustMeanAbsoluteDeviation',
 'pre_wavelet-HLH_glszm_GrayLevelNonUniformityNormalized',
 'pre_wavelet-LHH_firstorder_InterquartileRange',
 'pre_wavelet-LHH_glcm_Autocorrelation',
 'delta_log-sigma-1-0-mm-3D_glszm_LargeAreaLowGrayLevelEmphasis',
 'delta_log-sigma-1-5-mm-3D_firstorder_Energy',
 'delta_log-sigma-1-5-mm-3D_firstorder_TotalEnergy',
 'delta_log-sigma-1-5-mm-3D_gldm_DependenceVariance',
 'delta_log-sigma-2-0-mm-3D_ngtdm_Busyness',
 'delta_log-sigma-2-5-mm-3D_glszm_ZoneEntropy',
 'delta_original_shape_LeastAxisLength',
 'delta_square_glcm_Correlation',
 'delta_squa

In [ ]:
clinical_train[rad_features_list] = final_data[rad_features_list]
clinical_test[rad_features_list] = final_data[rad_features_list]
clinical_train

,encoded_PDL1,pre_log-sigma-1-5-mm-3D_glszm_GrayLevelNonUniformityNormalized,pre_log-sigma-1-5-mm-3D_glszm_SizeZoneNonUniformityNormalized,pre_log-sigma-2-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis,pre_original_shape_Maximum2DDiameterColumn,pre_squareroot_firstorder_Mean,pre_squareroot_glrlm_ShortRunLowGrayLevelEmphasis,pre_squareroot_glszm_LargeAreaHighGrayLevelEmphasis,pre_wavelet-HHH_firstorder_RobustMeanAbsoluteDeviation,pre_wavelet-HLH_glszm_GrayLevelNonUniformityNormalized,...,delta_square_glszm_LowGrayLevelZoneEmphasis,delta_wavelet-HHH_glszm_ZoneEntropy,delta_wavelet-HHL_firstorder_Median,delta_wavelet-HHL_firstorder_Uniformity,delta_wavelet-HLH_glcm_ClusterShade,delta_wavelet-HLH_glcm_JointEnergy,delta_wavelet-HLL_firstorder_Energy,delta_wavelet-LLH_glszm_GrayLevelNonUniformityNormalized,delta_wavelet-LLH_glszm_SmallAreaEmphasis,delta_wavelet-LLL_glszm_SizeZoneNonUniformityNormalized
pt_no,,,,,,,,,,,,,,,,,,,,,
51127213,1.0,0.654321,0.135802,0.926739,35.000000,3.987763,0.095471,2.371810e+06,0.001982,0.625000,...,0.000000,0.000000,0.000041,-0.000002,0.000037,-0.000504,509.711550,-0.104376,0.012771,0.001135
38924846,0.0,0.820000,0.300000,1.152795,32.310989,3.550385,0.055671,1.034499e+07,0.011696,0.555556,...,0.000000,0.584963,-0.000480,0.001067,0.028762,0.004004,-288.284702,0.214286,-0.074180,0.010441
56241796,1.0,0.802469,0.209877,1.638925,33.837849,3.517083,0.022095,2.080088e+08,0.012250,0.510204,...,0.000000,0.500000,0.000132,-0.000010,0.008949,0.000158,418.441321,-0.027874,-0.226132,-0.130211
40815882,1.0,0.520000,0.180000,0.732356,23.194827,2.990924,0.050635,2.315535e+07,0.006470,0.500000,...,0.000000,0.000000,0.000531,-0.000151,0.004207,-0.002678,31.474672,-0.045351,0.190010,-0.052852
33908849,0.0,0.625000,0.101562,0.972224,40.000000,3.534827,0.070666,2.343207e+07,0.008049,0.625000,...,0.202991,-0.736966,-0.000014,0.000010,0.014594,-0.000435,-777.711610,-0.029220,0.154312,-0.039461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47181472,2.0,0.500329,0.142669,0.578841,44.944410,3.147577,0.024669,3.249877e+09,0.010041,0.531250,...,0.000000,0.329447,-0.000140,0.000007,-0.000156,-0.002595,23.327664,-0.007686,0.088587,0.088412
49087653,1.0,0.680000,0.160000,1.103891,46.529560,4.234082,0.053637,2.994876e+07,0.012206,0.520000,...,0.000000,0.584963,0.000515,-0.000077,0.009308,0.001285,508.588955,0.064140,-0.135015,-0.022400
49588132,0.0,0.570312,0.156250,0.766630,55.072679,3.991733,0.312333,2.769617e+07,0.008017,0.510204,...,0.000000,0.072780,-0.000144,0.000032,0.010664,0.000594,431.863021,0.002959,-0.076182,0.092368


In [ ]:
train_data = clinical_train
test_data = clinical_test

In [ ]:
train_data

,encoded_PDL1,pre_log-sigma-1-5-mm-3D_glszm_GrayLevelNonUniformityNormalized,pre_log-sigma-1-5-mm-3D_glszm_SizeZoneNonUniformityNormalized,pre_log-sigma-2-5-mm-3D_glszm_SmallAreaHighGrayLevelEmphasis,pre_original_shape_Maximum2DDiameterColumn,pre_squareroot_firstorder_Mean,pre_squareroot_glrlm_ShortRunLowGrayLevelEmphasis,pre_squareroot_glszm_LargeAreaHighGrayLevelEmphasis,pre_wavelet-HHH_firstorder_RobustMeanAbsoluteDeviation,pre_wavelet-HLH_glszm_GrayLevelNonUniformityNormalized,...,delta_square_glszm_LowGrayLevelZoneEmphasis,delta_wavelet-HHH_glszm_ZoneEntropy,delta_wavelet-HHL_firstorder_Median,delta_wavelet-HHL_firstorder_Uniformity,delta_wavelet-HLH_glcm_ClusterShade,delta_wavelet-HLH_glcm_JointEnergy,delta_wavelet-HLL_firstorder_Energy,delta_wavelet-LLH_glszm_GrayLevelNonUniformityNormalized,delta_wavelet-LLH_glszm_SmallAreaEmphasis,delta_wavelet-LLL_glszm_SizeZoneNonUniformityNormalized
pt_no,,,,,,,,,,,,,,,,,,,,,
51127213,1.0,0.654321,0.135802,0.926739,35.000000,3.987763,0.095471,2.371810e+06,0.001982,0.625000,...,0.000000,0.000000,0.000041,-0.000002,0.000037,-0.000504,509.711550,-0.104376,0.012771,0.001135
38924846,0.0,0.820000,0.300000,1.152795,32.310989,3.550385,0.055671,1.034499e+07,0.011696,0.555556,...,0.000000,0.584963,-0.000480,0.001067,0.028762,0.004004,-288.284702,0.214286,-0.074180,0.010441
56241796,1.0,0.802469,0.209877,1.638925,33.837849,3.517083,0.022095,2.080088e+08,0.012250,0.510204,...,0.000000,0.500000,0.000132,-0.000010,0.008949,0.000158,418.441321,-0.027874,-0.226132,-0.130211
40815882,1.0,0.520000,0.180000,0.732356,23.194827,2.990924,0.050635,2.315535e+07,0.006470,0.500000,...,0.000000,0.000000,0.000531,-0.000151,0.004207,-0.002678,31.474672,-0.045351,0.190010,-0.052852
33908849,0.0,0.625000,0.101562,0.972224,40.000000,3.534827,0.070666,2.343207e+07,0.008049,0.625000,...,0.202991,-0.736966,-0.000014,0.000010,0.014594,-0.000435,-777.711610,-0.029220,0.154312,-0.039461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47181472,2.0,0.500329,0.142669,0.578841,44.944410,3.147577,0.024669,3.249877e+09,0.010041,0.531250,...,0.000000,0.329447,-0.000140,0.000007,-0.000156,-0.002595,23.327664,-0.007686,0.088587,0.088412
49087653,1.0,0.680000,0.160000,1.103891,46.529560,4.234082,0.053637,2.994876e+07,0.012206,0.520000,...,0.000000,0.584963,0.000515,-0.000077,0.009308,0.001285,508.588955,0.064140,-0.135015,-0.022400
49588132,0.0,0.570312,0.156250,0.766630,55.072679,3.991733,0.312333,2.769617e+07,0.008017,0.510204,...,0.000000,0.072780,-0.000144,0.000032,0.010664,0.000594,431.863021,0.002959,-0.076182,0.092368


In [ ]:
print('Data for Modeling: ' + str(train_data.shape))
print('Unseen Data For Predictions: ' + str(test_data.shape))

Data for Modeling: (79, 31)
Unseen Data For Predictions: (22, 31)


In [ ]:
ypCR_clf = setup(data=train_data,
            test_data=test_data,
            target='encoded_PDL1',
            normalize=True, 
            normalize_method='zscore', 
            data_split_shuffle=True, 
            data_split_stratify='encoded_PDL1',
            fold = 5,
            fold_strategy = 'stratifiedkfold',
            fold_shuffle = True, 
            session_id=3457, 
            use_gpu=True)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDAP=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDAP=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDAP=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDAP=1


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDAP=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDAP=1


,Description,Value
0,Session id,3457
1,Target,encoded_PDL1
2,Target type,Multiclass
3,Original data shape,"(101, 31)"
4,Transformed data shape,"(101, 31)"
5,Transformed train set shape,"(79, 31)"
6,Transformed test set shape,"(22, 31)"
7,Numeric features,30
8,Preprocess,True
9,Imputation type,simple


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1


[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDAP=1


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1


[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDAP=1


In [ ]:
train = get_config(variable="train_transformed")
train['encoded_PDL1'].value_counts()

1.0    36
0.0    22
2.0    21
Name: encoded_PDL1, dtype: int64

In [ ]:
test = get_config(variable="test_transformed")
test['encoded_PDL1'].value_counts()

1.0    10
2.0     6
0.0     6
Name: encoded_PDL1, dtype: int64

In [ ]:
compare_models(n_select=3, sort='AUC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.6325,0.7717,0.6325,0.6477,0.5908,0.3820,0.4274,0.1160
lr,Logistic Regression,0.5667,0.7476,0.5667,0.5751,0.5633,0.3174,0.3193,0.0720
lda,Linear Discriminant Analysis,0.6175,0.7193,0.6175,0.6259,0.6171,0.4067,0.4092,0.0620
ada,Ada Boost Classifier,0.6192,0.7172,0.6192,0.6562,0.5900,0.3682,0.4195,0.1580
rf,Random Forest Classifier,0.6075,0.7063,0.6075,0.6108,0.5915,0.3558,0.3685,0.3400
et,Extra Trees Classifier,0.6325,0.6948,0.6325,0.6735,0.6312,0.4061,0.4191,0.2560
gbc,Gradient Boosting Classifier,0.5075,0.6667,0.5075,0.5199,0.4964,0.2133,0.2150,0.5060
nb,Naive Bayes,0.5458,0.6332,0.5458,0.5786,0.5382,0.2710,0.2796,0.0600
dt,Decision Tree Classifier,0.5083,0.6078,0.5083,0.5177,0.5039,0.2286,0.2327,0.0600
lightgbm,Light Gradient Boosting Machine,0.4925,0.5904,0.4925,0.4676,0.4708,0.2035,0.2091,0.1300


[KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                      metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                      weights='uniform'),
 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=3457, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 LinearDiscriminantAnalysis(covariance_estimator=None, n_components=None,
                            priors=None, shrinkage=None, solver='svd',
                            store_covariance=False, tol=0.0001)]

In [ ]:
lightgbm = create_model('lightgbm', fold=5, round=4, cross_validation=True)

In [ ]:
prediction = predict_model(lightgbm, data=test_data)